## Hypothesis Function<br>
### Simple Linear Regression<br>
H(x) = Wx + b<br>
x라는 vertor와 W라는 matrix의 곱<br>
### Multivariable Linear Regression<br>
#### Hypothesis Function: Naive<br>
H(x) = w1x1 + w2x2 + w3x3 + b<br>
입력변수가 3개라면 weight도 3개

hypothesis = x1_train * w1 + x2_train * w2 + x3_train * w3 + b

In [38]:
# 위의 식은 단순한 hypothesis 정의
# 하지만 x가 길이 1000의 vector라면? -> 식이 매우 길어짐

#### Hypothesis Function: Matrix<br>
matmul()로 한번에 계산<br>
hypothesis = x_train.matmul(W) + b
<!--mm or @-->

## Full Code with torch.optim (1)

In [45]:
import torch
# x_train : 학생들의 퀴즈1,2,3 점수
# y_train : 학생들의 기말고사 점수 
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])

y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

# 모델 초기화
W = torch.zeros((3, 1), requires_grad = True)
b = torch.zeros(1, requires_grad = True)

# optimizer 설정
optimizer = torch.optim.SGD([W, b], lr=1e-5)

In [46]:
nb_epochs = 20
for epoch in range(nb_epochs + 1):
    hypothesis = x_train.matmul(W) + b # 5 x 3 과 3 x 1의 곱 -> 5 x 1
    cost = torch.mean((hypothesis - y_train)**2)
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    print('Epoch {:4d}/{} hypothesis: {} Cost: {:.6f}'.format(
        epoch, nb_epochs, hypothesis.squeeze().detach(), 
        cost.item()
    ))

Epoch    0/20 hypothesis: tensor([0., 0., 0., 0., 0.]) Cost: 29661.800781
Epoch    1/20 hypothesis: tensor([67.2578, 80.8397, 79.6523, 86.7394, 61.6605]) Cost: 9298.520508
Epoch    2/20 hypothesis: tensor([104.9128, 126.0990, 124.2466, 135.3015,  96.1821]) Cost: 2915.712646
Epoch    3/20 hypothesis: tensor([125.9942, 151.4381, 149.2133, 162.4896, 115.5097]) Cost: 915.040527
Epoch    4/20 hypothesis: tensor([137.7968, 165.6247, 163.1911, 177.7112, 126.3307]) Cost: 287.936005
Epoch    5/20 hypothesis: tensor([144.4044, 173.5674, 171.0168, 186.2332, 132.3891]) Cost: 91.371017
Epoch    6/20 hypothesis: tensor([148.1035, 178.0144, 175.3980, 191.0042, 135.7812]) Cost: 29.758139
Epoch    7/20 hypothesis: tensor([150.1744, 180.5042, 177.8508, 193.6753, 137.6805]) Cost: 10.445305
Epoch    8/20 hypothesis: tensor([151.3336, 181.8983, 179.2240, 195.1707, 138.7440]) Cost: 4.391228
Epoch    9/20 hypothesis: tensor([151.9824, 182.6789, 179.9928, 196.0079, 139.3396]) Cost: 2.493135
Epoch   10/20 hypo

### nn.Module<br>
W와 b를 일일이 초기화 해주는 것은 모델이 커질수록 귀찮은 일이 될수 있다.<br>
1. nn.Linear()에 입력차원과 출력차원을 입력해준다.<br>
2. Hypothesis 계산은 forward()에서 처리한다.

### F.mse_loss<br>
torch.nn.functional에서 제공하는 loss function을 사용한다.

## Full Code with torch.optim (2)

In [49]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MultivariateLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 1)
        
    def forward(self, x):
        return self.linear(x)
    
    
# x_train : 학생들의 퀴즈1,2,3 점수
# y_train : 학생들의 기말고사 점수 
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])

y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

# 모델 초기화
model = MultivariateLinearRegressionModel()

# optimizer 설정
optimizer = torch.optim.SGD(model.parameters(), lr=1e-5)

In [50]:
nb_epochs = 20
for epoch in range(nb_epochs + 1):
    hypothesis = model(x_train)
    cost = F.mse_loss(hypothesis, y_train)
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    print('Epoch {:4d}/{} hypothesis: {} Cost: {:.6f}'.format(
        epoch, nb_epochs, hypothesis.squeeze().detach(), 
        cost.item()
    ))

Epoch    0/20 hypothesis: tensor([14.9085, 25.6099, 21.2488, 23.7558, 20.8729]) Cost: 22748.216797
Epoch    1/20 hypothesis: tensor([73.8074, 96.4009, 91.0009, 99.7140, 74.8683]) Cost: 7132.474121
Epoch    2/20 hypothesis: tensor([106.7831, 136.0339, 130.0527, 142.2403, 105.0982]) Cost: 2237.765869
Epoch    3/20 hypothesis: tensor([125.2452, 158.2228, 151.9164, 166.0493, 122.0225]) Cost: 703.533813
Epoch    4/20 hypothesis: tensor([135.5818, 170.6454, 164.1572, 179.3791, 131.4975]) Cost: 222.632568
Epoch    5/20 hypothesis: tensor([141.3691, 177.6001, 171.0105, 186.8420, 136.8020]) Cost: 71.894333
Epoch    6/20 hypothesis: tensor([144.6095, 181.4936, 174.8475, 191.0203, 139.7716]) Cost: 24.645065
Epoch    7/20 hypothesis: tensor([146.4240, 183.6733, 176.9957, 193.3596, 141.4339]) Cost: 9.833952
Epoch    8/20 hypothesis: tensor([147.4401, 184.8934, 178.1985, 194.6694, 142.3643]) Cost: 5.190468
Epoch    9/20 hypothesis: tensor([148.0093, 185.5763, 178.8721, 195.4027, 142.8849]) Cost: 3.7